In [1]:
import os
import sys
import random
from PIL import Image
import numpy as np
import pandas as pd


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import random
import time

sys.path.insert(1, '//workspace/dl2020')
# #from model_loader_CP2 import *
# from CP_helper_RCNN import *
# from data_helper_RCNN import *


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.utils.data


 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
#from engine import train_one_epoch, evaluate


from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box

from model_loader_bbox import *
from bbox import *


%load_ext autoreload
%autoreload 2
%load_ext autotime

In [2]:
#re save models

# checkpoint = torch.load("fastRCNN_autoencode14epoch.pt")
# bbox_model = fr50_Model()
# bbox_model.load_state_dict(checkpoint["model_state_dict"])

time: 355 µs


In [3]:
# torch.save({
#         "bbox_state_dict": bbox_model.state_dict(),
#         "road_state_dict": None#road_model.state_dict(),
   
#            }, "musketeer.pt")

time: 457 µs


In [4]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);
image_folder = '../data'
annotation_csv = '../data/annotation.csv'

time: 1.2 ms


In [5]:
#make a simple validation loader
labeled_scene_index = np.array([132, 133])
#copied from run_test.py

labeled_trainset_task1 = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform_task1(),
    extra_info=False
    )
dataloader_task1 = torch.utils.data.DataLoader(
    labeled_trainset_task1,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )

time: 99.4 ms


In [6]:
model_loader = ModelLoader()

time: 1.74 s


In [11]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0

with torch.no_grad():
    for i, data in enumerate(dataloader_task1):
        total += 1
        sample, target, road_image = data
        sample = sample.cuda()
        
        predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu() #does this solve our problem of first element
        if i%10 ==0:
            print(predicted_bounding_boxes.shape, target['bounding_box'][0].shape)
            
        #ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes, target['bounding_box'][0])
        #total_ats_bounding_boxes += ats_bounding_boxes
    

torch.Size([100, 2, 4]) torch.Size([20, 2, 4])
torch.Size([100, 2, 4]) torch.Size([19, 2, 4])
torch.Size([100, 2, 4]) torch.Size([30, 2, 4])
torch.Size([100, 2, 4]) torch.Size([32, 2, 4])
torch.Size([100, 2, 4]) torch.Size([34, 2, 4])
torch.Size([100, 2, 4]) torch.Size([38, 2, 4])
torch.Size([100, 2, 4]) torch.Size([37, 2, 4])
torch.Size([100, 2, 4]) torch.Size([33, 2, 4])
torch.Size([100, 2, 4]) torch.Size([34, 2, 4])
torch.Size([100, 2, 4]) torch.Size([36, 2, 4])
torch.Size([100, 2, 4]) torch.Size([35, 2, 4])
torch.Size([100, 2, 4]) torch.Size([24, 2, 4])
torch.Size([100, 2, 4]) torch.Size([20, 2, 4])
torch.Size([100, 2, 4]) torch.Size([6, 2, 4])
torch.Size([100, 2, 4]) torch.Size([6, 2, 4])
torch.Size([100, 2, 4]) torch.Size([7, 2, 4])
torch.Size([100, 2, 4]) torch.Size([6, 2, 4])
torch.Size([100, 2, 4]) torch.Size([6, 2, 4])
torch.Size([100, 2, 4]) torch.Size([5, 2, 4])
torch.Size([100, 2, 4]) torch.Size([3, 2, 4])
torch.Size([100, 2, 4]) torch.Size([8, 2, 4])
torch.Size([100, 2, 4

In [8]:
predicted_bounding_boxes

tensor([[[ 3.4236,  3.4236,  2.9915,  2.9915],
         [-0.6878, -0.8930, -0.6878, -0.8930]],

        [[-2.4128, -2.4128, -2.8653, -2.8653],
         [ 0.6976,  0.4924,  0.6976,  0.4924]],

        [[ 3.2719,  3.2719,  2.8232,  2.8232],
         [ 2.3748,  2.1718,  2.3748,  2.1718]],

        [[-0.1121, -0.1121, -0.5483, -0.5483],
         [ 3.3146,  3.1099,  3.3146,  3.1099]],

        [[ 3.2720,  3.2720,  2.8229,  2.8229],
         [ 2.2948,  2.0923,  2.2948,  2.0923]],

        [[ 3.2696,  3.2696,  2.8198,  2.8198],
         [ 2.4544,  2.2509,  2.4544,  2.2509]],

        [[ 0.0521,  0.0521, -0.3789, -0.3789],
         [ 2.9867,  2.7841,  2.9867,  2.7841]],

        [[ 3.1875,  3.1875,  2.7386,  2.7386],
         [ 2.3345,  2.1298,  2.3345,  2.1298]],

        [[ 0.0519,  0.0519, -0.3846, -0.3846],
         [ 3.3129,  3.1075,  3.3129,  3.1075]],

        [[ 3.3513,  3.3513,  2.9000,  2.9000],
         [ 2.3340,  2.1337,  2.3340,  2.1337]],

        [[ 3.3530,  3.3530,  2.9012,  2.

time: 9.86 ms


In [9]:
len(predicted_bounding_boxes)

100

time: 1.71 ms


In [10]:
target["bounding_box"].shape

torch.Size([1, 15, 2, 4])

time: 2.5 ms
